In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 03 - Validation et Test du Modèle\n",
    "\n",
    "Validation approfondie du modèle de production et tests de robustesse."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import joblib\n",
    "import json\n",
    "from pathlib import Path\n",
    "from sklearn.metrics import (\n",
    "    classification_report, confusion_matrix, \n",
    "    precision_recall_curve, average_precision_score,\n",
    "    roc_curve, roc_auc_score, calibration_curve\n",
    ")\n",
    "from sklearn.model_selection import learning_curve\n",
    "import shap\n",
    "import lime\n",
    "from lime.lime_tabular import LimeTabularExplainer\n",
    "\n",
    "# Configuration\n",
    "plt.style.use('ggplot')\n",
    "np.random.seed(42)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Chargement du modèle et des données\n",
    "model_dir = Path('../models')\n",
    "model = joblib.load(model_dir / 'random_forest.pkl')\n",
    "\n",
    "# Charger les métadonnées\n",
    "with open(model_dir / 'model_metadata.json', 'r') as f:\n",
    "    metadata = json.load(f)\n",
    "\n",
    "print(\"=== MODÈLE CHARGÉ ===\")\n",
    "print(f\"Modèle: {metadata['model_name']}\")\n",
    "print(f\"Date d'entraînement: {metadata['training_date']}\")\n",
    "print(f\"F1-Score: {metadata['metrics']['f1_score']:.3f}\")\n",
    "print(f\"Features: {metadata['feature_names']}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Recréer le dataset de test\n",
    "df = pd.read_csv('../data/sample_data.csv')\n",
    "\n",
    "# Appliquer le même preprocessing qu'à l'entraînement\n",
    "def prepare_features(data):\n",
    "    df_prep = data.copy()\n",
    "    if 'department' in df_prep.columns:\n",
    "        from sklearn.preprocessing import LabelEncoder\n",
    "        le = LabelEncoder()\n",
    "        df_prep['department_encoded'] = le.fit_transform(df_prep['department'])\n",
    "        df_prep = df_prep.drop('department', axis=1)\n",
    "    \n",
    "    df_prep['creativity_burnout_ratio'] = df_prep['creative_score'] / (df_prep['burnout_scale'] + 1)\n",
    "    df_prep['high_creativity'] = (df_prep['creative_score'] > df_prep['creative_score'].quantile(0.75)).astype(int)\n",
    "    df_prep['high_burnout'] = (df_prep['burnout_scale'] > df_prep['burnout_scale'].quantile(0.75)).astype(int)\n",
    "    \n",
    "    return df_prep\n",
    "\n",
    "df_features = prepare_features(df)\n",
    "\n",
    "# Séparer features et target\n",
    "exclude_cols = ['employee_id', 'adhd_risk', 'autism_risk']\n",
    "feature_cols = [col for col in df_features.columns if col not in exclude_cols]\n",
    "X = df_features[feature_cols]\n",
    "y = df_features['adhd_risk']\n",
    "\n",
    "print(f\"Dataset de validation: {X.shape}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Validation complète du modèle\n",
    "predictions = model.predict(X)\n",
    "probabilities = model.predict_proba(X)[:, 1]\n",
    "\n",
    "# Métriques détaillées\n",
    "from sklearn.metrics import (\n",
    "    accuracy_score, precision_score, recall_score, f1_score,\n",
    "    balanced_accuracy_score, matthews_corrcoef\n",
    ")\n",
    "\n",
    "print(\"=== MÉTRIQUES DE VALIDATION ===\")\n",
    "print(f\"Accuracy: {accuracy_score(y, predictions):.3f}\")\n",
    "print(f\"Balanced Accuracy: {balanced_accuracy_score(y, predictions):.3f}\")\n",
    "print(f\"Precision: {precision_score(y, predictions):.3f}\")\n",
    "print(f\"Recall: {recall_score(y, predictions):.3f}\")\n",
    "print(f\"F1-Score: {f1_score(y, predictions):.3f}\")\n",
    "print(f\"Matthews Correlation: {matthews_corrcoef(y, predictions):.3f}\")\n",
    "print(f\"AUC-ROC: {roc_auc_score(y, probabilities):.3f}\")\n",
    "print(f\"Average Precision: {average_precision_score(y, probabilities):.3f}\")\n",
    "\n",
    "# Rapport de classification détaillé\n",
    "print(\"\\n=== RAPPORT DE CLASSIFICATION ===\")\n",
    "print(classification_report(y, predictions, target_names=['No ADHD Risk', 'ADHD Risk']))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Visualisations avancées\n",
    "fig, axes = plt.subplots(2, 3, figsize=(18, 12))\n",
    "fig.suptitle('Analyse de Performance du Modèle', fontsize=16, fontweight='bold')\n",
    "\n",
    "# 1. Matrice de confusion\n",
    "cm = confusion_matrix(y, predictions)\n",
    "sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],\n",
    "            xticklabels=['No Risk', 'Risk'],\n",
    "            yticklabels=['No Risk', 'Risk'])\n",
    "axes[0,0].set_title('Matrice de Confusion')\n",
    "axes[0,0].set_ylabel('Vraie Classe')\n",
    "axes[0,0].set_xlabel('Classe Prédite')\n",
    "\n",
    "# 2. Courbe ROC\n",
    "fpr, tpr, _ = roc_curve(y, probabilities)\n",
    "auc = roc_auc_score(y, probabilities)\n",
    "axes[0,1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {auc:.3f})')\n",
    "axes[0,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')\n",
    "axes[0,1].set_xlim([0.0, 1.0])\n",
    "axes[0,1].set_ylim([0.0, 1.05])\n",
    "axes[0,1].set_xlabel('Taux de Faux Positifs')\n",
    "axes[0,1].set_ylabel('Taux de Vrais Positifs')\n",
    "axes[0,1].set_title('Courbe ROC')\n",
    "axes[0,1].legend(loc=\"lower right\")\n",
    "axes[0,1].grid(alpha=0.3)\n",
    "\n",
    "# 3. Courbe Precision-Recall\n",
    "precision, recall, _ = precision_recall_curve(y, probabilities)\n",
    "avg_precision = average_precision_score(y, probabilities)\n",
    "axes[0,2].plot(recall, precision, color='red', lw=2, label=f'AP = {avg_precision:.3f}')\n",
    "axes[0,2].set_xlabel('Rappel')\n",
    "axes[0,2].set_ylabel('Précision')\n",
    "axes[0,2].set_title('Courbe Precision-Recall')\n",
    "axes[0,2].legend()\n",
    "axes[0,2].grid(alpha=0.3)\n",
    "\n",
    "# 4. Distribution des probabilités\n",
    "axes[1,0].hist(probabilities[y==0], bins=20, alpha=0.7, label='No Risk', color='lightblue')\n",
    "axes[1,0].hist(probabilities[y==1], bins=20, alpha=0.7, label='Risk', color='salmon')\n",
    "axes[1,0].set_xlabel('Probabilité Prédite')\n",
    "axes[1,0].set_ylabel('Fréquence')\n",
    "axes[1,0].set_title('Distribution des Probabilités')\n",
    "axes[1,0].legend()\n",
    "axes[1,0].axvline(x=0.5, color='red', linestyle='--', label='Seuil')\n",
    "\n",
    "# 5. Calibration plot\n",
    "fraction_of_positives, mean_predicted_value = calibration_curve(y, probabilities, n_bins=10)\n",
    "axes[1,1].plot(mean_predicted_value, fraction_of_positives, \"s-\", label=\"Modèle\")\n",
    "axes[1,1].plot([0, 1], [0, 1], \"k:\", label=\"Calibration parfaite\")\n",
    "axes[1,1].set_ylabel('Fraction de positifs')\n",
    "axes[1,1].set_xlabel('Probabilité moyenne prédite')\n",
    "axes[1,1].set_title('Courbe de Calibration')\n",
    "axes[1,1].legend()\n",
    "axes[1,1].grid(alpha=0.3)\n",
    "\n",
    "# 6. Feature importance\n",
    "if hasattr(model, 'feature_importances_'):\n",
    "    importance_df = pd.DataFrame({\n",
    "        'feature': X.columns,\n",
    "        'importance': model.feature_importances_\n",
    "    }).sort_values('importance', ascending=True).tail(10)\n",
    "    \n",
    "    axes[1,2].barh(importance_df['feature'], importance_df['importance'], alpha=0.7)\n",
    "    axes[1,2].set_title('Top 10 Features Importantes')\n",
    "    axes[1,2].set_xlabel('Importance')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Analyse SHAP pour l'explicabilité\n",
    "print(\"=== ANALYSE SHAP ===\")\n",
    "explainer = shap.TreeExplainer(model)\n",
    "shap_values = explainer.shap_values(X.iloc[:100])  # Échantillon pour rapidité\n",
    "\n",
    "# Si classification binaire, prendre la classe positive\n",
    "if len(shap_values) == 2:\n",
    "    shap_values = shap_values[1]\n",
    "\n",
    "# Summary plot\n",
    "plt.figure(figsize=(10, 8))\n",
    "shap.summary_plot(shap_values, X.iloc[:100], plot_type=\"bar\", show=False)\n",
    "plt.title('Importance des Features (SHAP)')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Waterfall plot pour un exemple\n",
    "plt.figure(figsize=(10, 6))\n",
    "shap.waterfall_plot(explainer.expected_value[1], shap_values[0], X.iloc[0], show=False)\n",
    "plt.title('Explication SHAP - Employé Exemple')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Test de robustesse avec données synthétiques\n",
    "print(\"=== TESTS DE ROBUSTESSE ===\")\n",
    "\n",
    "def generate_synthetic_data(n_samples=100):\n",
    "    \"\"\"Générer des données synthétiques pour tester la robustesse.\"\"\"\n",
    "    synthetic_data = pd.DataFrame({\n",
    "        'creative_score': np.random.normal(75, 20, n_samples),\n",
    "        'burnout_scale': np.random.randint(1, 11, n_samples),\n",
    "        'creativity_burnout_ratio': np.random.uniform(5, 20, n_samples),\n",
    "        'high_creativity': np.random.choice([0, 1], n_samples),\n",
    "        'high_burnout': np.random.choice([0, 1], n_samples)\n",
    "    })\n",
    "    \n",
    "    # Ajouter autres colonnes si nécessaires\n",
    "    for col in X.columns:\n",
    "        if col not in synthetic_data.columns:\n",
    "            synthetic_data[col] = np.random.choice([0, 1], n_samples)\n",
    "    \n",
    "    return synthetic_data[X.columns]  # Même ordre de colonnes\n",
    "\n",
    "# Test avec données synthétiques\n",
    "synthetic_X = generate_synthetic_data(200)\n",
    "synthetic_predictions = model.predict(synthetic_X)\n",
    "synthetic_probabilities = model.predict_proba(synthetic_X)[:, 1]\n",
    "\n",
    "print(f\"Prédictions sur données synthétiques:\")\n",
    "print(f\"- Pourcentage de cas à risque: {synthetic_predictions.mean()*100:.1f}%\")\n",
    "print(f\"- Probabilité moyenne: {synthetic_probabilities.mean():.3f}\")\n",
    "print(f\"- Écart-type des probabilités: {synthetic_probabilities.std():.3f}\")\n",
    "\n",
    "# Test avec valeurs extrêmes\n",
    "extreme_cases = pd.DataFrame({\n",
    "    'creative_score': [100, 0, 50],\n",
    "    'burnout_scale': [10, 1, 5],\n",
    "    'creativity_burnout_ratio': [10, 0.1, 10],\n",
    "    'high_creativity': [1, 0, 0],\n",
    "    'high_burnout': [1, 0, 1]\n",
    "})\n",
    "\n",
    "# Compléter avec les autres colonnes (valeurs moyennes)\n",
    "for col in X.columns:\n",
    "    if col not in extreme_cases.columns:\n",
    "        extreme_cases[col] = X[col].mean()\n",
    "\n",
    "extreme_cases = extreme_cases[X.columns]  # Même ordre\n",
    "extreme_preds = model.predict(extreme_cases)\n",
    "extreme_probs = model.predict_proba(extreme_cases)[:, 1]\n",
    "\n",
    "print(\"\\nCas extrêmes:\")\n",
    "for i, (pred, prob) in enumerate(zip(extreme_preds, extreme_probs)):\n",
    "    case_type = ['Créatif épuisé', 'Non-créatif calme', 'Moyen stressé'][i]\n",
    "    print(f\"- {case_type}: Prédiction={pred}, Probabilité={prob:.3f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Test de performance temporelle\n",
    "import time\n",
    "\n",
    "print(\"=== TESTS DE PERFORMANCE ===\")\n",
    "\n",
    "# Test de latence\n",
    "latencies = []\n",
    "for _ in range(100):\n",
    "    sample = X.sample(n=1)\n",
    "    start_time = time.time()\n",
    "    _ = model.predict(sample)\n",
    "    latency = (time.time() - start_time) * 1000  # en millisecondes\n",
    "    latencies.append(latency)\n",
    "\n",
    "print(f\"Latence moyenne: {np.mean(latencies):.2f} ms\")\n",
    "print(f\"Latence médiane: {np.median(latencies):.2f} ms\")\n",
    "print(f\"Latence P95: {np.percentile(latencies, 95):.2f} ms\")\n",
    "print(f\"Latence P99: {np.percentile(latencies, 99):.2f} ms\")\n",
    "\n",
    "# Test de débit (throughput)\n",
    "batch_sizes = [1, 10, 100, 1000]\n",
    "throughputs = []\n",
    "\n",
    "for batch_size in batch_sizes:\n",
    "    batch_data = X.sample(n=batch_size)\n",
    "    start_time = time.time()\n",
    "    _ = model.predict(batch_data)\n",
    "    elapsed_time = time.time() - start_time\n",
    "    throughput = batch_size / elapsed_time\n",
    "    throughputs.append(throughput)\n",
    "    print(f\"Batch size {batch_size}: {throughput:.0f} prédictions/seconde\")\n",
    "\n",
    "# Visualiser les performances\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "# Histogramme des latences\n",
    "axes[0].hist(latencies, bins=20, alpha=0.7, color='skyblue')\n",
    "axes[0].axvline(np.mean(latencies), color='red', linestyle='--', label=f'Moyenne: {np.mean(latencies):.2f} ms')\n",
    "axes[0].set_xlabel('Latence (ms)')\n",
    "axes[0].set_ylabel('Fréquence')\n",
    "axes[0].set_title('Distribution des Latences')\n",
    "axes[0].legend()\n",
    "\n",
    "# Courbe de débit\n",
    "axes[1].plot(batch_sizes, throughputs, 'o-', color='green', linewidth=2, markersize=8)\n",
    "axes[1].set_xlabel('Taille du Batch')\n",
    "axes[1].set_ylabel('Prédictions/seconde')\n",
    "axes[1].set_title('Débit vs Taille du Batch')\n",
    "axes[1].grid(alpha=0.3)\n",
    "axes[1].set_xscale('log')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Rapport final de validation\n",
    "validation_report = {\n",
    "    'model_performance': {\n",
    "        'f1_score': f1_score(y, predictions),\n",
    "        'precision': precision_score(y, predictions),\n",
    "        'recall': recall_score(y, predictions),\n",
    "        'accuracy': accuracy_score(y, predictions),\n",
    "        'auc_roc': roc_auc_score(y, probabilities),\n",
    "        'average_precision': average_precision_score(y, probabilities)\n",
    "    },\n",
    "    'performance_metrics': {\n",
    "        'mean_latency_ms': np.mean(latencies),\n",
    "        'p95_latency_ms': np.percentile(latencies, 95),\n",
    "        'max_throughput_per_sec': max(throughputs)\n",
    "    },\n",
    "    'robustness_tests': {\n",
    "        'synthetic_data_risk_rate': synthetic_predictions.mean(),\n",
    "        'extreme_cases_handled': True,\n",
    "        'probability_distribution_stable': synthetic_probabilities.std() < 0.5\n",
    "    },\n",
    "    'validation_date': pd.Timestamp.now().isoformat(),\n",
    "    'validation_status': 'PASSED'\n",
    "}\n",
    "\n",
    "# Sauvegarder le rapport\n",
    "with open(model_dir / 'validation_report.json', 'w') as f:\n",
    "    json.dump(validation_report, f, indent=2)\n",
    "\n",
    "print(\"=== RAPPORT FINAL DE VALIDATION ===\")\n",
    "print(json.dumps(validation_report, indent=2))\n",
    "print(f\"\\n✅ Rapport sauvegardé: {model_dir / 'validation_report.json'}\")\n",
    "print(f\"🎯 Statut de validation: {validation_report['validation_status']}\")\n",
    "print(f\"🚀 Modèle prêt pour la production !\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
